In [1]:
from train import train

CFG = { # TODO: make it a class
    'seed': 42,
    'cv_fold': 5,
    'base_model': 'resnet34',   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, efficientnet_v2_s, convnext_tiny, swin_t
    'batch_size': 32,
    'epochs': 3,
    'base_lr': 1e-3,
    'affine_degrees': 0,
    'affine_translate': None,
    'affine_scale': None,
    'dataloader_num_workers': 8,
    'scheduler_step_size': 2,
    'optimizer': 'AdamW',
    'scheduler': 'StepLR',
    'loss': 'KLDivLoss',
    'lr_gamma': 0.1,
    'sgd_momentum': 0.9,
    'color_jitter': {'brightness': 0, 'contrast': 0, 'saturation': 0, 'hue': 0},
    'random_erasing_p': 0,
    'freeze_epochs': 0,
    'in_channels': 1
}

tags=['torch', 'cv', 'best_epoch']
notes = ''
train_final_model = True
use_wandb = True
one_fold = False

for base_model in ['resnet34']:
    CFG['base_model'] = base_model
    for epochs in [3, 4]:
        CFG['epochs'] = epochs
        for scheduler in ['StepLR', 'CosineAnnealingLR', 'OneCycleLR']:
            CFG['scheduler'] = scheduler
            for spec_trial_selection in ['mean_offset', 'all']:
                CFG['spec_trial_selection'] = spec_trial_selection
                print(f'##########################\nTraining with epochs {epochs}, scheduler {scheduler}, base_model: {base_model}, spec_trial_selection: {spec_trial_selection}\n##########################\n')
                train(CFG, tags=tags, notes=notes, train_final_model=train_final_model, use_wandb=use_wandb, one_fold=one_fold)

AttributeError: 'dict' object has no attribute 'spec_trial_selection'

In [ ]:
# for base_model in ['resnet34']:
#     CFG['base_model'] = base_model
#     for epochs in [3, 4]:
#         CFG['epochs'] = epochs
#         for scheduler in ['StepLR', 'CosineAnnealingLR', 'OneCycleLR']:
#             CFG['scheduler'] = scheduler
#             print(f'##########################\nTraining with epochs {epochs}, scheduler {scheduler}, base_model: {base_model}\n##########################\n')
#             train(CFG, tags=tags, notes=notes, train_final_model=train_final_model, use_wandb=use_wandb, one_fold=one_fold)